In [97]:
from bs4 import BeautifulSoup
import pandas as pd
import pickle
from statistics import mean

In [98]:
html_dict = pickle.load(open("./dataout/HTMLlocation_dict.pk", "rb"))
table = pd.read_csv("capitals.csv")

cities = list(table["capital"])
city_price={}
city_price_cur = []
price_values=[]

for city in cities:
    try:
        test_city = html_dict[city]
        soup = BeautifulSoup(test_city, "html.parser")
        price_list = soup.find_all("div" , attrs = {'data-target' : "price"})

        for i in range(0,len(price_list)):
            price_list[i]=price_list[i].text
            price_list[i]=price_list[i].replace(",","")
            price_list[i]=price_list[i].replace("*","")
            currency = price_list[0][0]
            price_values.append(int(price_list[i][1:]))

        city_price = {'city':city, 'price':round(mean(price_values),2), 'currency':currency}
        city_price_cur.append(city_price)
    except:
        print(city)


In [104]:
mean_price = pd.DataFrame(city_price_cur)
mean_price = mean_price.drop("currency",axis=1)
mean_price["price"] = round(mean_price["price"]*1.02,2)
mean_price

,city,price
0,Kabul,229.50
1,Tirana,82.28
2,Algiers,77.87
3,Andorra la Vella,77.18
4,Luanda,101.99
...,...,...
192,Caracas,136.34
193,Hanoi,136.04
194,Sana'a,136.04
195,Lusaka,135.89


In [105]:
acommodation = pd.merge(left = mean_price,
                                right = table,
                                how = "inner",
                                left_on = "city",
                                right_on = "capital")
acommodation

,city,price,Unnamed: 0,capital
0,Kabul,229.50,Afghanistan,Kabul
1,Tirana,82.28,Albania,Tirana
2,Algiers,77.87,Algeria,Algiers
3,Andorra la Vella,77.18,Andorra,Andorra la Vella
4,Luanda,101.99,Angola,Luanda
...,...,...,...,...
192,Caracas,136.34,Venezuela,Caracas
193,Hanoi,136.04,Vietnam,Hanoi
194,Sana'a,136.04,Yemen,Sana'a
195,Lusaka,135.89,Zambia,Lusaka


In [106]:
acommodation = acommodation.drop("city",axis=1)
acommodation.rename(columns = {'Unnamed: 0':'country'}, inplace = True)

In [110]:
acommodation = acommodation[["country","capital","price"]]
acommodation

,country,capital,price
0,Afghanistan,Kabul,229.50
1,Albania,Tirana,82.28
2,Algeria,Algiers,77.87
3,Andorra,Andorra la Vella,77.18
4,Angola,Luanda,101.99
...,...,...,...
192,Venezuela,Caracas,136.34
193,Vietnam,Hanoi,136.04
194,Yemen,Sana'a,136.04
195,Zambia,Lusaka,135.89


In [111]:
acommodation.to_csv("acommodation.csv",index=False)